In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np

In [ ]:
class AirModel_GRU(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super().__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.linear(x)
        return x

In [ ]:
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i:i + lookback]
        target = dataset[i + 1:i + lookback + 1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')

lookback = 3
X_train, y_train = create_dataset(train_data, lookback)
X_test, y_test = create_dataset(test_data, lookback)

In [ ]:
hidden_size = 100
num_layers = 1
n_epochs = 10

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = AirModel_GRU(hidden_size, num_layers).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

loader = DataLoader(TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

for epoch in tqdm(range(n_epochs), desc="Training epochs"):
    model.train()
    for batch in loader:
        X, y = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), 'model.pth')

model.eval()
with torch.no_grad():
    predictions = model(X_test.to(device))
    print("Predictions:", predictions.cpu().numpy())